In [7]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2 as cv
import numpy as np
import joblib
from mtcnn import MTCNN
from keras_facenet import FaceNet

# Load pre-trained model, encoder, and embeddings
model = joblib.load(r"C:\Users\Admin\Downloads\svm_model_1_160x160.pkl")
encoder = joblib.load(r"C:\Users\Admin\Downloads\label_encoder_new_dataset.pkl")

# Load embeddings and labels if necessary (in case you need to check or do something with them)
data = np.load(r"C:\Users\Admin\Downloads\faces_embeddings_done_4classes1.npz")
EMBEDDED_X = data['arr_0']
Y = data['arr_1']

# Function to extract embeddings for a new image
def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D (160x160x3)
    face_img = np.expand_dims(face_img, axis=0)  # 4D (None, 160, 160, 3)
    embedder = FaceNet()  # Create an embedder object
    yhat = embedder.embeddings(face_img)
    return yhat[0]  # 512D embedding (1x1x512)

# Function to predict from an image path
def predict_from_image(image_path):
    t_im = cv.imread(image_path)
    t_im = cv.cvtColor(t_im, cv.COLOR_BGR2RGB)

    detector = MTCNN()
    faces = detector.detect_faces(t_im)

    if len(faces) > 0:
        x, y, w, h = faces[0]['box']
        x, y = abs(x), abs(y)
        t_im = t_im[y:y+h, x:x+w]
        t_im = cv.resize(t_im, (160, 160))

        test_im = get_embedding(t_im)
        test_im = [test_im]  # SVM expects 2D array

        # Predict using the SVM model
        ypreds = model.predict(test_im)

        # Inverse transform the prediction to get the label
        predicted_class = encoder.inverse_transform(ypreds)
        return f"Predicted class: {predicted_class[0]}"
    else:
        return "No face detected"

# GUI setup using Tkinter
class FaceRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Face Recognition System")
        self.root.geometry("600x600")
        
        # Label for displaying predictions
        self.result_label = tk.Label(root, text="Upload an image to predict", font=("Helvetica", 12))
        self.result_label.pack(pady=10)
        
        # Button for uploading image
        self.upload_button = tk.Button(root, text="Upload Image", command=self.upload_image, font=("Helvetica", 12))
        self.upload_button.pack(pady=10)
        
        # Label to display the uploaded image
        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)
        
    def upload_image(self):
        # Open file dialog to upload an image
        image_path = filedialog.askopenfilename(title="Select an image", filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
        
        if image_path:
            # Display the uploaded image in the GUI
            img = Image.open(image_path)
            img.thumbnail((250, 250))
            img = ImageTk.PhotoImage(img)
            self.image_label.config(image=img)
            self.image_label.image = img
            
            # Predict the face identity
            result = predict_from_image(image_path)
            
            # Update the result label with the prediction
            self.result_label.config(text=result)

# Create the Tkinter window and run the app
root = tk.Tk()
app = FaceRecognitionApp(root)
root.mainloop()


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.6.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 [==============================] - 2s 2s/step
